In [1]:
import pandas as pd
OUTPUT_PERSONS_PATH = "C:/Users/user/PTV_Intern/src/DeepAM/dataset/persons_info.csv"
hh_df = pd.read_csv("C:/Users/user/PTV_Intern/data/raw/NHTS_US_2017/hhpub.csv")
per_df = pd.read_csv("C:/Users/user/PTV_Intern/data/raw/NHTS_US_2017/perpub.csv")
trip_df = pd.read_csv("C:/Users/user/PTV_Intern/data/raw/NHTS_US_2017/trippub.csv")
hh_df = hh_df[hh_df['HHSIZE']<=5]
per_df = per_df[per_df['HHSIZE']<=5]
print(len(hh_df), len(per_df))

127848 253644


In [2]:
# --- Attributes of Interestfor each dataframe ---
nhts_household_columns = [
    'HOUSEID',      # (인덱스용)
    'HHFAMINC',     # 1. Household Income
    'HHSIZE',       # 2. Household Size
    'HHVEHCNT',     # 3. Household Vehicles
    'HOMEOWN',      # 4. Home Ownership Status
    'STDCNT',       # 5. Household Students (SCHTYP로 계산)
    'DRVRCNT',      # 6. Household Licensed Drivers
    'LIF_CYC',      # 7. Household Life Cycle Stage
    'WRKCOUNT',     # 8. Household Employed Members
    'URBRUR',       # 9. Type of Residence (URBAN은 4개 분류, URBRUR은 2개 분류)
    'HBRESDN',      # 10. Housing Density
    'HBPPOPDN',     # 11. Population Density
    'HBHTNRNT',     # 12. Renter-Occupied Housing Ratio
    # '',           # (없음) Housing Status
]
nhts_person_columns = [
    'HOUSEID',      # (인덱스용)
    'PERSONID',     # (인덱스용)
    'DRIVER',       # 1. Driver's License Status
    'EDUC',         # 2. Education Level
    'OCCAT',        # 3. Job Category
    'R_SEX_IMP',    # 4. Gender
    'R_AGE_IMP',    # 5. Age
    'R_RACE',       # 6. Racial/Ethnic Identity
    'PTUSED',       # 7. PTUSED
    'R_RELAT',      # 8. Household Role
    'GT1JBLWK',     # 9. (Family에선 제외) Number of Jobs
    'WORKER',       # 10. Employment Status
    'WRK_HOME',     # 11. (Family에선 제외) Employment Location Type
    # 'GRADE',        # (없음) Current School Grade Level
    # '',             # (없음) Number of Workdays
]
# Other Household Members Info. 블럭 구성시 필요한 9개 feature
family_attributes = [
    'HOUSEID',      # (인덱스용)
    'PERSONID',     # (인덱스용)
    'DRIVER',       # 1. Driver's License Status
    'EDUC',         # 2. Education Level
    'OCCAT',        # 3. Job Category
    'R_SEX_IMP',    # 4. Gender
    'R_AGE_IMP',    # 5. Age
    'R_RACE',       # 6. Racial/Ethnic Identity
    'PTUSED',       # 7. PTUSED
    'R_RELAT',      # 8. Household Role
    'WORKER',       # 9. Employment Status
]

In [3]:
# Calculate SCHTYP attribute from persons
per_df['IS_STUDENT'] = (per_df['SCHTYP'] == 1).astype(int)
stdcnt_series = per_df.groupby('HOUSEID')['IS_STUDENT'].sum()
hh_df['STDCNT'] = hh_df['HOUSEID'].map(stdcnt_series)

# Drop unusing columns
hh_df = hh_df[nhts_household_columns]
per_df = per_df[nhts_person_columns]

print(len(hh_df.columns), len(per_df.columns))
combined_df = per_df.merge(hh_df, on='HOUSEID', how='left')
len(combined_df.columns)

13 13


25

In [4]:
# Redefine Indice starting from 0 (conventional procedure to make equal interval between codes)
att_dict = dict()
for col in combined_df.columns:
    if col in ['HOUSEID', 'PERSONID']: continue
    label_dict = dict()
    print(f"{col}: {combined_df[col].unique()}", end='')
    
    # -7이하는 99로 변환, -1은 98로 변환
    combined_df[col] = combined_df[col].apply(lambda x: 99 if x<=-7 else x)
    combined_df[col] = combined_df[col].apply(lambda x: 98 if x==-1 else x)
    
    # 오름차순 정렬 => [유효 값, ..., Appropriate skip,결측치] 순으로 정렬됨
    unique_vals = sorted(combined_df[col].unique())

    # 1부터 시작하는 label로 변환
    for new, old in enumerate(unique_vals):
        label_dict[old] = new + 1
    combined_df[col] = combined_df[col].apply(lambda x: label_dict[x])
    print(f"{combined_df[col].unique()}")
    
    # 0-based label => original label 로 복원하기 위한 dictionary
    att_dict[col] = dict()
    for old in label_dict.keys():
        new = label_dict[old]
        att_dict[col][new] = old

DRIVER: [ 1  2 -1][1 2 3]
EDUC: [ 3  2  5  4  1 -1 -8 -7][3 2 5 4 1 6 7]
OCCAT: [-1  2  4  3 -9  1 97 -8 -7][6 2 4 3 7 1 5]
R_SEX_IMP: [2 1][2 1]
R_AGE_IMP: [67 66 28 55 49 45 68 72 82 78 44 40 50 39 38 30 33 75 61 59 22 54 57 25
 29  6  5 52 56 86 69 87 31 76 51 17 64 37  7 63 65 24 46 14 23 58 60 73
 85 18 19 77 81 47 42 15 13 92 35 62 70 12 36 26 41 16 11 34 48 53 10  8
 74 32 20  9 79 43 27 71 21 83 80 84 88][63 62 24 51 45 41 64 68 78 74 40 36 46 35 34 26 29 71 57 55 18 50 53 21
 25  2  1 48 52 82 65 83 27 72 47 13 60 33  3 59 61 20 42 10 19 54 56 69
 81 14 15 73 77 43 38 11  9 85 31 58 66  8 32 22 37 12  7 30 44 49  6  4
 70 28 16  5 75 39 23 67 17 79 76 80 84]
R_RACE: [ 2  1  6 97  3  4  5 -7 -8][2 1 6 7 3 4 5 8]
PTUSED: [ 0 20  5 22 27  1 21  2 30 10  8  3 18 15 25  6  4 12  7 16 24 17 11 -8
 -7 28 19 13  9 14 23 26 29 -9][ 1 21  6 23 28  2 22  3 31 11  9  4 19 16 26  7  5 13  8 17 25 18 12 32
 29 20 14 10 15 24 27 30]
R_RELAT: [ 1  2  3  7  5  4  6 -9 -7 -8][1 2 3 7 5 4 6 8]
G

In [5]:
combined_df.to_csv(OUTPUT_PERSONS_PATH)

In [9]:
person_ids_df = pd.read_csv('C:/Users/user/PTV_Intern/src/DeepAM/dataset/person_ids.csv', index_col=0)
all_person_df = pd.read_csv(OUTPUT_PERSONS_PATH, index_col=0)